**CYK.PY**

Funktionalitäten werden aus `eingabe.py` importiert, dazu eine neue Grammatik eingelesen und in der Variable `grammar` gespeichert

In [ ]:
import eingabe

grammar = eingabe.cfg
eingabe.cfg.new_grammar(grammar)

`check_rule` sucht für einen gegebenen String `rhs` (right-hand-side) alle passenden Symbole `lhs` raus und gibt deren Position in `grammar.rules` mittels `indices` zurück.

In [ ]:
def check_rule(rhs):
    indices = []
    for lhs in grammar.rules:
        if lhs.find(rhs) != -1:
            indices.append(grammar.rules.index(lhs))
        elif lhs.find(rhs[::1]) != -1 and len(rhs) > 1:
            indices.append(grammar.rules.index(lhs))
    return indices

`matrix_mult` bildet alle Kombinationen von zwei gegebenen Feldern `left`, `lower` in der Tabelle, fügt die passenden Symbole `symbols` hinzu und gibt diese zurück.

In [ ]:
def matrix_mult(left, lower):
    symbols = []
    for symbol_left in left:
        for symbol_lower in lower:
            symbols.append(grammar.rules[x][0] for x in check_rule(symbol_left + symbol_lower))
    flat = [x for inner in symbols for x in inner]
    return flat

`tableau` wird initialisiert. In `cyk` wird zunächst alle diagonalen Felder `tableau[i][i]` mit dem passenden Symbol gefüllt.
$\\$ 
Anschließend wird die obere Dreiecksmatrix durch Aufrufe von `matrix_mult` befüllt, mit `horizontal` und `vertical` sind dabei die Einträge aus `tableau` links und unter dem aktuell zu Befüllenden gemeint.

In [1]:

tableau = []


def cyk(word):

    n = len(word)
    for i in range(0, n):
        tableau.append([""] * n)
        tableau[i][i] = [grammar.variables[x] for x in check_rule(word[i])]

    for s in range(1, n):
        for i in range(1, n - s + 1):
            result = []
            for k in range(i, i + s):
                horizontal = tableau[i - 1][k - 1]
                vertical = tableau[k][i + s - 1]
                result += (matrix_mult(horizontal, vertical))
            tableau[i - 1][i + s - 1] = list(dict.fromkeys(result))
    return tableau


In [ ]:
**EINGABE.PY**

Das Package `re` zur Benutzung von Regulären Ausdrücken wird importiert, sowie eine Hilfsliste zum teilen von Strings initialisiert. $\\$
`new_word` nimmt einen string als Nutzereingabe und gibt diesen als Charakterliste `chars` zurück.

In [ ]:
import re

splitter = ['.', ';', '  ', ',']

def new_word():
        word = input("what is the word?\n")
        chars = []
        for c in word:
            chars.append(c)
        return chars


Die Klasse der Kontextfreien Grammatik `cfg` mitsamt Konstruktor und Setter-Methoden.

In [ ]:
class cfg:
    
    def __init__(self):
        self.variables = []
        self.alphabet = []
        self.rules = []
        self.start = None

    def set_variables(self, variables):
        self.variables = variables

    def set_alphabet(self, alphabet):
        self.alphabet = alphabet

    def set_rules(self, rules):
        self.rules = rules

    def set_start(self, start):
        self.start = start

In der Methode `new_grammar` werden Symbole, Alphabet, Regeln und das Startsymbol eingelesen. Außerdem werden kleine Syntaktische Korrekturen mithilfe der `.split`-Methode vorgenommen, genauso wird durch die Methode `check_syntax` der Fall abgefangen, dass die eingegebene Grammatik nicht dem vorgegebenen Syntax entspricht.

In [ ]:
def new_grammar(self):

        # Variabeln input
        var = input("Bitte geben sie alle Variablen ein.\n")
        for i in range(0, len(splitter)):
            var = var.replace(splitter[i], ' ')
        cfg.set_variables(cfg, var.split())

        # Aplhabet input
        var = input("Bitte geben sie das Alphabet an.\n")
        for i in range(0, len(splitter)):
            var = var.replace(splitter[i], ' ')
        cfg.set_alphabet(self, var.split())

        # Regeln input
        var = input("Bitte geben sie die Regeln der Grammatik an.\n")
        var = var.replace(' ', '')

        # cfg.set_rules(self, re.split(';|,|.', var))
        cfg.set_rules(self, re.split(';|,', var))

        # Start input
        cfg.set_start(self, input("Bitte geben sie die start Variable an.\n"))
        if cfg.check_syntax(self, self.variables, self.alphabet, self.rules, self.start) != 1:
            print(
                "Es gibt syntaktische Fehler in der Grammatik, bitte beheben sie diese und Probieren sie es noch einmal.\n")
            raise SystemExit

-- methode zum Syntax überprüfen-- $\\$
 @Simon you're up i dont know what you did here tbh :D

**TABULAR.PY**

Funktionalitäten zur Stringmanipulation und Erstellung des `LaTeX`-Tableaus werden importiert.

In [ ]:
from string import Template
import cyk
import tabulate

Nachdem der CYK-Algorithmus durchgelaufen ist, wird `table` an `to_latex` übergeben, durch das Package `tabulate` zu einem String in LaTeX Format transformiert und per `Template` in einen vorgefertigten String eingefügt. 
Zuletzt wird geprüft, ob $w \in L$ und dementsprechend unter der tabelle vermerkt.

In [ ]:
def to_latex(table,length_word):
    v_indices = (str(x + 1) for x in range(length_word))
    h_indices = (str(x + 1) for x in range(length_word))

    latex_string = str(tabulate.tabulate(table, tablefmt="latex", showindex=iter(v_indices), headers=iter(h_indices)))
    latex_string = latex_string.replace("  1 & ", r"\hline \n  1 & ", 1)
    latex_string = latex_string.replace("[]", r"$\emptyset$")
    latex_string = Template(r'\documentclass[12pt]{article}\n \usepackage{threeparttable}\n \begin{document} \n $table\n \begin{tablenotes}\item[1] $wL$\end{tablenotes}\end{document}%').safe_substitute(table=latex_string)
    latex_string = latex_string.replace(r'\begin{tabular}{r"+"l"*n',r'\begin{tabular}{|r"+"|l"*n+"|')
    latex_string = latex_string.replace("['", r"\{")
    latex_string = latex_string.replace("']", r"\}")
    latex_string = latex_string.replace("', '", ", ")
    
    if table[0][-1] == []:
        latex_string = latex_string.replace(r"$wL$", r"$w \notin L$")
        return latex_string
    
    elif table[0][-1][0].find(cyk.grammar.rules[0][0]) != -1:
        latex_string = latex_string.replace("$wL$", r"$w \in L$")
        return latex_string
    
    else:
        latex_string = latex_string.replace(r"$wL$", r"$w \notin L$")
        return latex_string

In [ ]:
def check_syntax(self, variables, alphabet, rules, start):
        lower = []
        upper = []
        wrong = 0
        for i in range(0, len(rules)):
             if rules[i].find(start, 0, 1) == -1:
                wrong += 1
            var = rules[i]
            for j in range(0, len(var)):
                if var[j].islower():
                    lower.append(var[j])
                if var[j].isupper():
                    upper.append(var[j])
        if wrong == len(rules):
            print(1)
            return -1
        # Checkt ob alle benutzen Buchstaben in den Regeln auch im Alphabet gegeben sind
        for i in range(0, len(lower)):
            wrong = 0
            for j in range(0, len(alphabet)):
                if lower[i] != alphabet[j]:
                    wrong += 1
            if wrong == len(alphabet):
                print(2)
                return -1
        # Checkt ob alle benutzen Variablen in den Regeln auch gegeben sind
        for i in range(0, len(upper)):
            wrong = 0
            for j in range(0, len(variables)):
                if upper[i] != variables[j]:
                    wrong += 1
            if wrong == len(variables):
                print(3)
                return -1
        return 1